In [ ]:
# imports
from openeo import connect, Connection
from openeo.rest.datacube import DataCube
from typing import List, Dict, Tuple, Union
import pathlib
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline

In [ ]:
DEFAULT_FIGSIZE = (9, 6)

def show_tiff(filename:str, figsize=DEFAULT_FIGSIZE, vmin=None, vmax=None, rescale_percentile=97, add_colorbar=False):
    """Small helper to load a geotiff and visualize it"""
    with rasterio.open(filename) as ds:
        data = ds.read()

    fig, ax = plt.subplots(figsize=figsize)

    if len(data.shape) == 3:
        if data.max() > 500:
            p = np.percentile(data, rescale_percentile, axis=[1,2])
            data = data / p[:, None, None]
            data = np.clip(data, 0, 1)        
        data = np.moveaxis(data, 0, 2)
        if data.shape[2]<3:
            data = data[:,:,0]
        
    im = ax.imshow(data, vmin=vmin, vmax=vmax,cmap="RdYlGn")
    if add_colorbar:
        fig.colorbar(im, ax=ax, fraction=0.05)

out_dir = pathlib.Path("output")
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Connect to VITO backend
vito_url: str = "https://openeo.vito.be/openeo/1.0"
vito_creo_url: str = "https://openeo.creo.vito.be"
ee_url: str = "https://earthengine.openeo.org"
incd_url: str = "https://openeo.a.incd.pt/openeo/"

# con: Connection = connect(ee_url).authenticate_basic(username="group1", password="test123")
# con: Connection = connect(vito_creo_url)
# con.authenticate_oidc(provider_id="egi")
con: Connection = connect(incd_url).authenticate_oidc(provider_id="egi")

In [ ]:
cube: DataCube = con.datacube_from_process("mean", data=[2, 3, 5, 8])
cube.execute()

In [ ]:
con.list_collections()


In [ ]:
bbox = {
    'west': 5.017099,
    'south': 51.219585,
    'east': 5.023468,
    'north': 51.223363,
    'crs': 'EPSG:4326',
}

s2: DataCube = con.load_collection("SENTINEL2_L1C",
    spatial_extent=bbox,
    temporal_extent=["2017-01-01", "2017-04-01"],
    bands=["B02", "B03", "B04"]).rename_labels(dimension="bands", target=["R", "G", "B"], source=["B04", "B03", "B02"]
)

# s2 = s2.reduce_dimension(dimension="t", reducer="median")

s2.download(out_dir / "s2.tiff", format="GTiff")
# s2.download(out_dir / "s2.json", format="JSON")
# s2.download(out_dir / "s2.png", format="png")

s2.graph
    

In [ ]:
import rasterio

dataset: rasterio.DatasetReader = rasterio.open(out_dir / "s2.tiff")
plt.imshow(dataset.read(1))
plt.show()